In [1]:
abstract MicroLang = {
    cat
    Utt ;
    S ;
    VP ;
    AP ;
    CN ; -- common noun (without determiner)
    NP ; -- noun phrase
    Det ;
    V ;
    A ; 
    N ;

    fun
-- Phrase
    UttS    : S  -> Utt ;
    UttNP   : NP -> Utt ;

-- Sentence
    PredVPS : NP -> VP -> S ;

-- Verb
    UseV    : V  -> VP ;
    
-- Noun
    DetCN      : Det -> CN -> NP ;
    a_Det      : Det ;
    aPl_Det    : Det ;
    the_Det    : Det ;
    thePl_Det  : Det ;
    UseN       : N -> CN ;
    AdjCN      : AP -> CN -> CN ;
    
-- Adjective
    PositA     : A -> AP ;

    fun
    -- boy_N : N ;
    -- girl_N : N ;
    house_N : N ;
    clever_A : A ;
    big_A : A ;
    jump_V : V ;
    play_V : V ;
    
}


Defined MicroLang

In [2]:
resource MicroResSwe = {

    param
        Number = Sg | Pl ;
        Species = Indef | Def ;
        Gender = Utr | Neutr ;
    
        Degree = Positive | Comparative | Superlative ;

    oper
    -- Noun
        Noun : Type = { s : Number => Species => Str ; g : Gender} ;

        mkNoun : Str -> Str -> Str -> Str -> Gender -> Noun
        = \sgIndef,sgDef,plIndef,plDef,gen -> {
          s = table {
              Sg => table {Indef => sgIndef ; Def => sgDef} ;
              Pl => table {Indef => plIndef; Def => plDef}
          } ;
          g = gen
        } ;

        smartNoun : Str -> Gender -> Noun = \sgIndef,gen -> case gen of {
        Utr => case sgIndef of {
            pojk + "e" => mkNoun sgIndef (sgIndef + "en") (pojk + "ar") (pojk + "arna") gen ;
            flick + "a" => mkNoun sgIndef (flick + "an") (flick + "or") (flick + "orna") gen ;
            cyk + "el" => mkNoun sgIndef (cyk + "eln") (cyk + "lar") (cyk + "larna") gen ;
            vä + "n" => mkNoun sgIndef (vä + "nnen") (vä + "nner") (vä + "nnerna") gen ;
            dat + "or" => mkNoun sgIndef (dat + "orn") (dat + "orer") (dat + "orerna") gen ;
            _ => mkNoun sgIndef (sgIndef + "en") (sgIndef + "ar") (sgIndef + "arna") gen
            } ;
        Neutr => case sgIndef of {
            äppl + "e" => mkNoun sgIndef (äppl + "et")  (äppl + "en") (äppl + "ena") gen ;
            vatt + "en" => mkNoun sgIndef (vatt + "net")  (vatt + "en") (vatt + "nen") gen ;
            _ => mkNoun sgIndef (sgIndef + "et") (sgIndef) (sgIndef + "en") gen
            }
        } ;
    
    -- Adjective
        Adjective : Type = {s : Number => Species => Gender => Str} ;
        
        mkAdjective : Str -> Str -> Str -> Str -> Adjective
        = \utrSgIndef, neutrSgIndef, sgDef, pl -> {
          s = table {
              Sg => table {
                  Indef => table {Utr => utrSgIndef ; Neutr => neutrSgIndef}; 
                  Def => table {_ => sgDef}
              } ;
              Pl => table {
                    _=> table {
                        _ => pl
                    }
                }
            }
        } ;
 
        smartAdjective : Str -> Adjective = \adj -> case adj of {
            smar + "t" => mkAdjective adj adj (adj + "a") (adj + "a") ;
            _ => mkAdjective adj (adj + "t") (adj + "a") (adj + "a")
        } ;
  
    -- Verb
        Verb : Type = {s : Str} ;
    
        be_Verb : Verb = {s = "är"} ;
        
}

Abstract changed, previous concretes discarded.


In [3]:
concrete MicroLangSwe of MicroLang = open MicroResSwe in {
    lincat
    Utt = {s : Str} ;
    
    S = {s : Str} ;
    VP = {s : Verb} ;
    AP = Adjective ;
    CN = Noun ;
    NP = {s : Str} ;
    Det = {s : Gender => Str ; n : Number ; sp : Species ;} ;
    V = Verb ;
    A = Adjective ; 
    N = Noun ;
    
    lin
-- Phrase
    UttS s = s ;
    UttNP np = {s = np.s} ;

-- Sentence
    --  NP -> VP -> S ;
    PredVPS np vp = {
        s = np.s ++ vp.s.s
    } ;

-- Verb
    UseV v = {
        s = v ;
    };
    
-- Noun
    -- Det -> CN -> NP
    DetCN det cn = {
        s = det.s ! cn.g ++ cn.s ! det.n ! det.sp ;
    } ;

    a_Det = {
        s = table {
            Utr => "en" ;
            Neutr => "ett"
        } ;
        n = Sg ; 
        sp = Indef
    } ;
    aPl_Det = {
        s = table { _ => "flera" } ; 
        n = Pl ;
        sp = Indef
    } ;
    the_Det = {
        s = table {
            Utr => "den" ;
            Neutr => "det"
        } ;
        n = Sg ; 
        sp = Def
    } ;
    thePl_Det = {
        s = table { _ => "de" } ;
        n = Pl ;
        sp = Def
    } ;
    
    UseN n = n ;
    -- Noun : Type = { s : Number => Species => Str ; g : Gender} ;
    -- Adjective : Type = { s : Number => Species => Gender => Str}
    AdjCN ap cn = {
        g = cn.g ;
        s = table {
            n => table {
                sp => ap.s ! n ! sp ! cn.g ++ cn.s ! n ! sp
            }
        } ;
    } ;
    
-- Adjective
    PositA a = a;
    
    -- lin boy_N = mkN "pojke" Utr ;
    -- lin girl_N = mkN "flicka" Utr ;
    lin house_N = mkN "hus" Neutr ;
    lin clever_A = mkA "smart" ;
    lin big_A = mkA "stor" ;
    lin jump_V = mkV "hoppa" ;
    lin play_V = mkV "leka" ;

    oper 
    mkN = overload {
        mkN : Str -> Gender -> Noun   -- predictable noun, e.g. bil-bilen-bilar-bilarna, fluga-flugan-flugor-flugorna
          = \n,g -> lin N (smartNoun n g) ;
        mkN : Str -> Str -> Str -> Str -> Gender -> Noun  -- irregular noun, e.g. man-mannen-män-männen,
          = \sgIndef,sgDef,plIndef,plDef,gen -> lin N (mkNoun sgIndef sgDef plIndef plDef gen) ;
        } ;
    -- utrSgIndef, neutrSgIndef, sgDef, pl 
    mkA = overload {
        mkA : Str -> A -- predictable adjective
          = \adj -> lin A (smartAdjective adj) ;
        mkA : Str -> Str -> Str -> Str -> A -- very irregular adjective
          = \utrSgIndef,neutrSgIndef,sgDef,pl -> lin A (mkAdjective utrSgIndef neutrSgIndef sgDef pl) ;
    } ;
    
    mkV : Str -> Verb
        = \s -> lin V {s = s} ;
}


Abstract changed, previous concretes discarded.


In [4]:
gr

PredVPS (DetCN aPl_Det (AdjCN (PositA clever_A) (UseN house_N))) (UseV jump_V)


In [5]:
gr | lin

de smarta husen hoppa


In [6]:
gt | lin

flera stora hus hoppa
flera stora hus leka
flera smarta hus hoppa
flera smarta hus leka
flera hus hoppa
flera hus leka
ett stort hus hoppa
ett stort hus leka
ett smart hus hoppa
ett smart hus leka
ett hus hoppa
ett hus leka
de stora husen hoppa
de stora husen leka
de smarta husen hoppa
de smarta husen leka
de husen hoppa
de husen leka
det stora huset hoppa
det stora huset leka
det smarta huset hoppa
det smarta huset leka
det huset hoppa
det huset leka


Exported MicroLang.gf

Exported MicroResSwe.gf

Exported MicroLangSwe.gf

Exported MicroLangSwe.gf